In [61]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

In [62]:
df = pd.read_csv('/Users/burak/Developer/MachineLearning/4462 - Final Project/notebooks/mock_user_data.csv')

In [63]:
df.sample(3)

,username,country,title,date_watched,percentage_watched,rating
10120,Kenneth Lyons,Georgia,['Freedomland' 'Tyson' 'Acts of Violence' 'Mys...,2002.0,91.0,6.0
12318,Jeffrey Clark,Singapore,['Scooby-Doo on Zombie Island' 'Riverdale' 'Yo...,2014.0,49.0,10.0
1668,Kathy White,Micronesia,['Postcards from London' '\u200bMaj Rati \u200...,2003.0,37.0,7.0


In [64]:
df.shape

(15001, 6)

In [65]:
df_titles = df['title'].str.get_dummies(sep=',')

# remove if there is a [ or ] in the game name
df_titles = df_titles.rename(columns=lambda x: x.replace('[', '').replace(']', ''))

In [66]:
#replace title column with the new columns
df = pd.concat([df, df_titles], axis=1)
df.drop(['title'], inplace=True, axis=1)

df.sample(3)

,username,country,date_watched,percentage_watched,rating,15'\n 'Budhia Singh: Born to Run' 'Bangkok Love Stories: Innocence'\n 'Remember Me' 'Steve Jobs' 'The School of Mischief',"15' ""Eve's Apple"" 'Happy Jail' 'Thackeray (Hindi)'\n 'Hotel Pacific' 'Kota Factory' 'KO One Return' 'J-Style Trip'\n 'The Darkness'","15' ""Kevin Hart's Guide to Black History"" 'Disjointed'\n 'Hong Kong West Side Stories' 'Marvel Anime: Wolverine'\n 'Lo más sencillo es complicarlo todo' 'Secret Obsession' 'Action Replayy'",15' 'A Most Violent Year'\n 'Meet the Trumps: From Immigrant to President'\n 'All Hail King Julien: Exiled',15' 'Chris Rock: Tamborine'\n 'Day of Destiny' 'Bobby Robson: More Than a Manager'\n 'A Cinderella Story: Christmas Wish' 'Mere Papa Hero Hiralal'\n 'Hide and Seek',...,'الف مبروك' 'Oasis: Supersonic' 'They Are Everywhere'\n 'Full Out 2: You Got This!' 'Shikara' 'KO One Return' 'Get Shorty',"'دفعة القاهرة'\n ""Bruno and Boots: This Can't Be Happening at Macdonald Hall"" 'Joker'\n 'Motu Patlu: Deep Sea Adventure' '9' 'Kay Dyache Bola'\n 'The Royal House of Windsor' ""Angela's Christmas"" 'Cop Car'","'دفعة القاهرة' ""Alice Doesn't Live Here Anymore"" 'One on One' 'The Mist'\n 'The Sinking Of Van Der Wijck' 'Love O2O'\n 'A Cinderella Story: Christmas Wish' 'Rip Tide'",'دفعة القاهرة' 'Alias JJ,'دفعة القاهرة' 'Liar,"'دفعة القاهرة' 'ReMastered: Who Shot the Sheriff'\n 'Let It Fall: Los Angeles 1982-1992' 'Taken' ""Garfield's Pet Force""\n 'Flock of Four' 'When Hari Got Married'",'يوم الدين' 'Maya Angelou: And Still I Rise' 'Anthony Kaun Hai?'\n 'Laddaland' 'The Could’ve-Gone-All-the-Way Committee' 'My MVP Valentine'\n 'My Little Pony Equestria Girls: Forgotten Friendship'\n 'Maine Pyaar Kyun Kiya' 'John Mulaney: Kid Gorgeous at Radio City',"'海的儿子' 'Kay Dyache Bola' 'Memoir of a Murderer' 'My MVP Valentine'\n ""Who's That Knocking at My Door?"" 'Violet Evergarden: Special' 'Lavender'\n 'Paradise Beach'",'최강전사 미니특공대 : 영웅의 탄생' 'Lock,'최강전사 미니특공대 : 영웅의 탄생' 'The Workshop' 'La Piloto' 'Padamu Aku Bersujud'\n 'Siren'
760,Tina Adams,Ukraine,2020.0,67.0,7.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,Jessica Garcia,France,2013.0,15.0,5.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5375,Isabella Robinson,Tokelau,2022.0,28.0,9.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
from sklearn.preprocessing import LabelEncoder

In [68]:
encoder = LabelEncoder()
df['country'] = encoder.fit_transform(df['country'])

In [69]:
df_final = df.drop('username', axis=1)
df_final.dropna(inplace=True)

In [70]:
#print number of null
df_final.isnull().sum()

country                                                                                                                                                                                                                                                                      0
date_watched                                                                                                                                                                                                                                                                 0
percentage_watched                                                                                                                                                                                                                                                           0
rating                                                                                                                                                                                     

In [71]:
# Scale the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_final)

# Create the model
kmeans = KMeans(n_clusters=3, random_state=42)

# Fit the model
kmeans.fit(df_scaled)

# Predict the labels
labels = kmeans.predict(df_scaled)

# Calculate the silhouette score
score = silhouette_score(df_scaled, labels)
print('Silhouette Score: %.3f' % score)

/Users/burak/miniforge3/envs/we-hear-you/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Silhouette Score: 0.040


In [72]:
labels = pd.DataFrame(labels, columns=['cluster'])
df_labeled = pd.concat([df_final, labels], axis=1)

df_labeled.shape

(15001, 16400)

In [73]:
#print the label count
print(df_labeled['cluster'].value_counts())

cluster
2.0    5950
1.0    4575
0.0    4474
Name: count, dtype: int64


In [81]:
#assign ussername back to the dataframe
df_labeled['username'] = df['username']
df_labeled.dropna(inplace=True)

In [82]:
df_labeled['cluster']

2        1.0
3        2.0
4        2.0
5        0.0
6        1.0
        ... 
14994    0.0
14995    0.0
14996    2.0
14997    2.0
14998    1.0
Name: cluster, Length: 14997, dtype: float64

In [119]:
def get_recommendation(username):
    user_cluster = df_labeled[df_labeled['username'] == username]['cluster'].values[0]
    cluster_df = df_labeled[df_labeled['cluster'] == user_cluster]
    cluster_df = cluster_df[cluster_df['username'] != username]
    
    titles = []

    for i in range(4, 10):
        titles.append(cluster_df.columns[i])
    
    print(type(titles))



    return titles


list = get_recommendation('Mark Lopez')
print(list)

<class 'list'>
[" 15'\n 'Budhia Singh: Born to Run' 'Bangkok Love Stories: Innocence'\n 'Remember Me' 'Steve Jobs' 'The School of Mischief'", ' 15\' "Eve\'s Apple" \'Happy Jail\' \'Thackeray (Hindi)\'\n \'Hotel Pacific\' \'Kota Factory\' \'KO One Return\' \'J-Style Trip\'\n \'The Darkness\'', ' 15\' "Kevin Hart\'s Guide to Black History" \'Disjointed\'\n \'Hong Kong West Side Stories\' \'Marvel Anime: Wolverine\'\n \'Lo más sencillo es complicarlo todo\' \'Secret Obsession\' \'Action Replayy\'', " 15' 'A Most Violent Year'\n 'Meet the Trumps: From Immigrant to President'\n 'All Hail King Julien: Exiled'", " 15' 'Chris Rock: Tamborine'\n 'Day of Destiny' 'Bobby Robson: More Than a Manager'\n 'A Cinderella Story: Christmas Wish' 'Mere Papa Hero Hiralal'\n 'Hide and Seek'", " 15' 'Fairy Tail'"]


In [120]:
import pickle

In [ ]:
with(open('titles.pkl', 'wb')) as f:
    pickle.dump(df_labeled, f)